In [1]:
import os
os.chdir("C:/Users/zetru/OneDrive/Mémoire python/")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = 16, 9
plt.rcParams['axes.grid'] = True
import seaborn as sns
sns.set_theme(style="whitegrid")

In [2]:
df=pd.read_hdf('df_v5.h5','df_v5')

In [3]:
df['id'].nunique()

9470

In [4]:
df['nom_carburant'] = df['nom_carburant'].replace({'E10': 'SP95-E10', 'SP95': 'SP95-E5'})

In [5]:
df['type'] = df['type'].replace({'A': 'Autoroute', 'R': 'Route'})

In [6]:
df.columns

Index(['cp', 'id', 'nom_carburant', 'id_carburant', 'maj', 'valeur',
       'litre_brut', 'type', 'latitude', 'longitude', 'ville',
       'stations_rayon_5km', 'competition_geo'],
      dtype='object')

In [7]:
df['maj'] = pd.to_datetime(df['maj'])
df = df.sort_values(by=['id', 'nom_carburant', 'maj'])
df['prix_diff'] = df.groupby(['id', 'nom_carburant'])['maj'].diff().dt.days

resultats = []

carburants = df['nom_carburant'].unique()

for carburant in carburants:
    for type_station in ['Autoroute', 'Route']:
        df_type = df[(df['nom_carburant'] == carburant) & (df['type'] == type_station)]
        
        if not df_type.empty:
            stats_type = {
                'Carburant': carburant,
                'Type': type_station,
                'Nb obs.': df_type['id'].nunique(),
                'Durée moyenne (en jours)': df_type['prix_diff'].mean(),
                'Ecart-type': df_type['prix_diff'].std(),
                'Q25': df_type['prix_diff'].quantile(0.25),
                'Q50': df_type['prix_diff'].median(),
                'Q75': df_type['prix_diff'].quantile(0.75)
            }
            resultats.append(stats_type)

resultats_df = pd.DataFrame(resultats)

resultats_df = resultats_df[['Carburant', 'Type', 'Nb obs.', 'Durée moyenne (en jours)', 'Ecart-type', 'Q25', 'Q50', 'Q75']]

resultats_df = resultats_df.round(2)

resultats_df

,Carburant,Type,Nb obs.,Durée moyenne (en jours),Ecart-type,Q25,Q50,Q75
0,Gazole,Autoroute,436,1.59,1.63,1.0,1.0,1.0
1,Gazole,Route,9023,2.47,3.14,1.0,1.0,3.0
2,SP95-E10,Autoroute,431,1.61,1.77,1.0,1.0,1.0
3,SP95-E10,Route,6808,2.28,2.87,1.0,1.0,3.0
4,SP98,Autoroute,428,1.63,1.90,1.0,1.0,1.0
5,SP98,Route,7701,2.62,3.74,1.0,1.0,3.0
6,SP95-E5,Autoroute,43,3.27,3.02,1.0,2.0,4.0
7,SP95-E5,Route,4317,4.03,5.22,1.0,2.0,5.0
8,E85,Autoroute,161,1.35,2.53,1.0,1.0,1.0
9,E85,Route,3013,2.72,5.57,1.0,1.0,2.0


In [8]:
formatters = {col: '{:.2f}'.format for col in resultats_df.set_index(['Carburant', 'Type']).select_dtypes(include=['float', 'int']).columns}
resultats_df.set_index(['Carburant', 'Type']).to_latex('tex/tableau_majINDEX.tex',formatters=formatters)

In [9]:
df['type'] = df['type'].replace({'Autoroute': 'A', 'Route': 'R'})

In [10]:
df_a=df[df['type']=='A']
df_r=df[df['type']=='R']

In [11]:
df_corr=pd.DataFrame(df.groupby(['nom_carburant','maj','type'])[['valeur','litre_brut']].mean()).reset_index()
df_corr_a=pd.DataFrame(df_a.groupby(['nom_carburant','maj','type'])[['valeur','litre_brut']].mean()).reset_index()
df_corr_r=pd.DataFrame(df_r.groupby(['nom_carburant','maj','type'])[['valeur','litre_brut']].mean()).reset_index()

In [12]:
def f_correlation(x):
    x=x.iloc[:,-2:]
    return x.corr().iloc[0, 1]

In [13]:
tableau_a=df_corr_a.groupby(['nom_carburant','type']).apply(f_correlation).reset_index().rename(columns={0:'Autoroute'}).drop(columns=['type'])
tableau_a

,nom_carburant,Autoroute
0,E85,0.330473
1,GPLc,0.421298
2,Gazole,0.791620
3,SP95-E10,0.902743
4,SP95-E5,0.854859
5,SP98,0.897102


In [14]:
tableau_r=df_corr_r.groupby(['nom_carburant','type']).apply(f_correlation).reset_index().rename(columns={0:'Route'}).drop(columns=['type'])
tableau_r

,nom_carburant,Route
0,E85,0.455217
1,GPLc,0.519583
2,Gazole,0.778601
3,SP95-E10,0.909465
4,SP95-E5,0.904595
5,SP98,0.908698


In [15]:
tableau_correlation=pd.concat([tableau_a,tableau_r.iloc[:,-1]],axis=1)
tableau_correlation

,nom_carburant,Autoroute,Route
0,E85,0.330473,0.455217
1,GPLc,0.421298,0.519583
2,Gazole,0.791620,0.778601
3,SP95-E10,0.902743,0.909465
4,SP95-E5,0.854859,0.904595
5,SP98,0.897102,0.908698


In [16]:
tableau_correlation.to_latex('tex/tableau_corr.tex',index=False)

In [17]:
tableau_maj=df.groupby(['id','nom_carburant','type'])['valeur'].size()\
                .reset_index()\
                .groupby(['nom_carburant','type'])['valeur']\
                .describe().round(2).reset_index()
tableau_maj

,nom_carburant,type,count,mean,std,min,25%,50%,75%,max
0,E85,A,161.0,243.81,140.53,1.0,75.00,353.0,360.00,364.0
1,E85,R,3013.0,118.55,122.92,1.0,24.00,55.0,207.00,365.0
2,GPLc,A,319.0,165.40,139.69,1.0,24.00,131.0,340.00,364.0
3,GPLc,R,1129.0,122.49,137.96,1.0,10.00,36.0,251.00,365.0
4,Gazole,A,436.0,225.29,105.03,27.0,133.00,202.0,358.25,365.0
5,Gazole,R,9023.0,142.56,94.38,1.0,65.00,122.0,197.00,365.0
6,SP95-E10,A,431.0,221.58,108.24,20.0,127.00,202.0,358.00,364.0
7,SP95-E10,R,6808.0,150.84,96.57,1.0,75.00,131.5,212.00,365.0
8,SP95-E5,A,43.0,25.63,49.35,1.0,1.00,1.0,10.50,203.0
9,SP95-E5,R,4317.0,75.72,66.93,1.0,31.00,57.0,103.00,365.0


In [18]:
formatters = {col: '{:.2f}'.format for col in tableau_maj.select_dtypes(include=['float', 'int']).columns}
tableau_maj.to_latex('tex/tableau_maj.tex',index=False, formatters=formatters)

In [19]:
tableau_maj.to_latex('tex/tableau_maj.tex',index=False)